In [ ]:
import os
import shutil
import pandas as pd, numpy as np
import cv2, imutils
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from hyperopt.pyll.base import scope 
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import tensorflow as tf

In [ ]:
cwd = os.getcwd()
df = pd.read_csv(cwd + '/../data_csv/preprocessing_data.csv')

In [ ]:
df.groupby('emotion').count()

In [ ]:
dir_img = cwd + '/crop_images'
if os.path.exists(dir_img):
    shutil.rmtree(dir_img)
os.makedirs(dir_img)

dictionary = ['ANGER', 'CONTEMPT', 'DISGUST', 'FEAR', 'HAPPINESS',  'NEUTRAL', 'SADNESS', 'SURPRISE']
for emo in dictionary:
    os.makedirs(dir_img + "/" + emo)

In [ ]:
feature =  []
labels = []
error = []

for i, row in df.iterrows():
    image_path = cwd + '/../cleaned_image/' + row.image
    image = cv2.imread(image_path)

    # detect face with haarcascade
    face_cascade = cv2.CascadeClassifier('../haarcascade/haarcascade_frontalface_alt.xml')
    face = face_cascade.detectMultiScale(
        image,
        scaleFactor = 1.1,
        minNeighbors = 3,
        minSize = (175, 175),
        flags = cv2.CASCADE_SCALE_IMAGE
    )

    # check detect face
    if len(face) != 1:
        error.append(i)
        continue

    # scope face
    for (x, y, w, h) in face:
        roi_image = image[y:y+h, x:x+w]

    # resize face
    roi_image = cv2.resize(roi_image, (128, 128), interpolation = cv2.INTER_AREA)

    # wrtite image
    cv2.imwrite(os.path.join(dir_img, row.image), roi_image)

    # get feature
    pixels = roi_image.flatten()
    feature.append(pixels)

    # get label
    labels.append(row.emotion)
    
    if i > 0 and i % 1000 == 0:
        print('[INFO] processed {}/{}'.format(i, len(df)))

In [ ]:
df.drop(error, axis=0, inplace=True)

In [ ]:
df.groupby('emotion').count()

In [ ]:
df.to_csv(cwd + "/crop_data.csv", index=False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(feature), np.array(labels), test_size=0.1, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=0)

In [ ]:
lb = preprocessing.LabelBinarizer()
lb.fit(y_train)

y_train = lb.transform(y_train)
y_test = lb.transform(y_test)
y_val = lb.transform(y_val)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

In [ ]:
def create_model(output):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(11, activation='relu'))
    model.add(Dense(3, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(output, activation='softmax'))
    opt = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = create_model(8)
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_val, y_val,), epochs=100, verbose=2)